In [290]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

## Load data 

In [294]:
shop_behavior_path = "F:\\Competitions\\Tianchi\\data\\ccf_first_round_user_shop_behavior.csv"
evaluation_path = "F:\\Competitions\\Tianchi\\data\\evaluation_public.csv" 
shop_info_path = "F:\\Competitions\\Tianchi\\data\\ccf_first_round_shop_info.csv"


In [302]:
shop_behavior_df =pd.read_csv(shop_behavior_path,nrows = 60000) 

In [303]:

shop_info = pd.read_csv(shop_info_path) 

In [304]:
#wifi list to top n wifi set
def array2dict(wifi_infos):
    wifi = [tuple(x.split("|")[0:2]) for x  in wifi_infos.split(";")]
    wifi.sort(key = lambda x: x[1],reverse =False)
    sel = [y[0] for y in wifi[0:5]]
    return set(sel)
shop_behavior_df["user_wifi"]= shop_behavior_df["wifi_infos"].map(array2dict)

In [314]:
# shop info merge wifi list
shop_user = pd.merge(shop_info,shop_behavior_df,on = 'shop_id', how='left', suffixes=['_s','_u'])
shop_user.drop(['wifi_infos'],axis = 1,inplace= True)
def unionset(set_df):
    #groupby shop_id to new wifi_list
    base_set = set_df.iloc[0]['user_wifi']
    for j in range(1,set_df.shape[0]-1):
        base_set = base_set | set_df.iloc[j]['user_wifi']
    return base_set
# unionset(shop_user[shop_user['shop_id']=='s_411850'])
top_wifi = shop_user.groupby('shop_id').apply(unionset)

top_wifi_df = top_wifi.to_frame()
top_wifi_df.reset_index(inplace= True)
top_wifi_df.rename(index=str, columns={0: "shop_wifi"},inplace = True)
shop_info_new = pd.merge(shop_info,top_wifi_df)
print shop_info_new.shape



(8477, 7)


# extract Feature

In [327]:
user_shop_new = pd.merge(shop_behavior_df,shop_info_new,on = 'shop_id', how='left', suffixes=['_u','_s'])
user_shop_new.drop(['wifi_infos'],axis = 1,inplace=True)
print user_shop_new.shape
user_shop_new_deNA = user_shop_new.dropna()
print user_shop_new_deNA.shape
# user_shop_new

(60000, 12)
(60000, 12)


,user_id,shop_id,time_stamp,longitude_u,latitude_u,user_wifi,category_id,longitude_s,latitude_s,price,mall_id,shop_wifi


In [328]:

user_shop_new['longitude_delta']=abs(user_shop_new['longitude_s']-user_shop_new['longitude_u'])
user_shop_new['latitude_delta']=abs(user_shop_new['latitude_s']-user_shop_new['latitude_u'])
def intersection(df):
    res = df['user_wifi'] & df['shop_wifi'] 
    coef = len(res)/1.0/len(df['shop_wifi'])
    return coef
user_shop_new['wifi_coef']  = user_shop_new.apply(intersection,axis=1)

In [329]:
user_shop_feature = user_shop_new[['user_id','shop_id','category_id','price','mall_id','wifi_coef','longitude_delta','latitude_delta']]
user_shop_feature


,user_id,shop_id,category_id,price,mall_id,wifi_coef,longitude_delta,latitude_delta
0,u_376,s_2871718,c_38,42,m_1409,0.833333,0.000491,0.000146
1,u_376,s_2871718,c_38,42,m_1409,0.833333,0.000620,0.000076
2,u_1041,s_181637,c_30,40,m_4079,0.070423,0.000741,0.000420
3,u_1158,s_609470,c_27,49,m_6587,0.200000,0.000110,0.000094
4,u_1654,s_3816766,c_6,62,m_3005,0.192308,0.000251,0.000141
5,u_2733,s_3686420,c_34,40,m_3839,0.238095,0.000864,0.000232
6,u_2848,s_89874,c_49,62,m_1920,0.555556,0.000122,0.000175
7,u_3063,s_939447,c_28,40,m_2878,0.333333,0.000460,0.000074
8,u_3063,s_939447,c_28,40,m_2878,0.333333,0.000408,0.000056
9,u_3063,s_3698335,c_25,54,m_2878,0.294118,0.000040,0.000243


## Training

In [330]:
def setLabel(df):
    label = 0
    if df['wifi_coef']>0.5:
        label = 1
    return label
        
training =user_shop_feature
training['label'] = training.apply(setLabel,axis =1)

D:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [337]:
print training[training['label']==1]['label'].count()
print training[training['label']==0]['label'].count()

7556
52444


## Model

In [281]:
def logistic_regression_classifier(train_x, train_y):  
    from sklearn.linear_model import LogisticRegression  
    model = LogisticRegression(penalty='l2')  
    model.fit(train_x, train_y)  
    return model
def random_forest_classifier(train_x, train_y):  
    from sklearn.ensemble import RandomForestClassifier  
    model = RandomForestClassifier(n_estimators=50 ,oob_score=True)  
    model.fit(train_x, train_y)  
    return model  

In [331]:
all_inputs=training[['wifi_coef','longitude_delta','latitude_delta']].values
all_classes=training['label'].values
(X_train,X_test,y_train,y_test)= train_test_split(all_inputs,all_classes,train_size=0.75)

In [332]:
rfmodel = random_forest_classifier(X_train,y_train)
rfmodel.oob_score_

1.0

In [333]:
y_pred = rfmodel.predict(X_train)
print("RF Metrics : {0}".format(roc_auc_score(y_train, y_pred)))


RF Metrics : 1.0


## Validation

,shop_id,category_id,longitude_s,latitude_s,price,mall_id,user_id,time_stamp,longitude_u,latitude_u,new_wifi,top_wifi
1988,s_101222,c_51,121.327088,31.776542,57,m_4011,NaN,NaN,NaN,NaN,NaN,NaN


## Predict

nan

{'i'}